In [71]:
natural_numbers = function() {
    i = 0L
    function() {
        (i <<- i + 1L)
    }
}

N = natural_numbers()

In [72]:
even_seq = function(seq) {
    function() {
        x = seq()
        while (x %% 2 != 0) {
            x = seq()
        }
        x
    }
}

In [73]:
filter = function(seq, f) {
    force(seq)
    function() {
        x = seq()
        while (!f(x)) {
            x = seq()
        }
        x
    }
}

In [74]:
evens = filter(N, function(x) x %% 2 == 0)     

In [75]:
evens()
evens()
evens()

[1] 2

[1] 4

[1] 6

In [76]:
not_div = function(seq, k) {
    filter(seq, function(x) x %% k != 0)
}
odds = not_div(natural_numbers(), 2)
odds()
odds()
odds()

[1] 1

[1] 3

[1] 5

In [77]:
prime_sieve = function() {
    n = natural_numbers()
    n()  # start from 2 since 1 is not prime
    s = n
    function() {
        r = s()
        s <<- not_div(s, r)
        r
    }
}
primes = prime_sieve()
first_20_primes = purrr::map_int(1:20, ~ primes())
first_20_primes

 [1]  2  3  5  7 11 13 17 19 23 29 31 37 41 43 47 53 59 61 67 71

In [78]:
chunk = function(f, k) {
    # add initial NA that will be chopped off when first called
    ch = c(NA, purrr::map_int(1:(k - 1), ~ f()))  
    function() {
        (ch <<- dplyr::lead(ch, 1, default=f()))
    }
}

In [79]:
chunked = chunk(natural_numbers(), 3)
chunked()
chunked()
chunked()

[1] 1 2 3

[1] 2 3 4

[1] 3 4 5

In [80]:
f = function() {
    sample(1:20, 1)
}

In [81]:
cf = chunk(f, 3)
cf()

[1] 11 14 13

In [82]:
all_equal = function(v) {
    length(unique(v)) == 1
}
s = filter(cf, all_equal)
s()
s()

[1] 14 14 14

[1] 8 8 8

In [83]:
pluck = function(seq, k) {
    force(seq)
    function() {
        seq()[[k]]
    }
}

In [84]:
library(magrittr)

call_until = function(f, g) {
    while (1) {
        s = f %>% chunk(3) %>% filter(all_equal) %>% pluck(1)
        x = s()
        if (x == g()) {
            return(x)
        }
    }
}

In [85]:
i = 0
set.seed(1)
f = function() {
    i <<- i + 1
    s = sample(1:8, 1)
}
j = 0
g = function() {
    j <<- j + 1
    s = sample(1:5, 1)
}
                   
call_until(f, g)
i
j

[1] 5

[1] 137

[1] 2